In [3]:
#Libraries we used in our code
import pandas as pd
import numpy as np
import random

# Read input CSV files (given)
courses_df = pd.read_csv('courses.csv')
student_course_df = pd.read_csv('studentCourse.csv')
student_name_df = pd.read_csv('studentNames.csv')
teachers_df = pd.read_csv('Teachers.csv')

# Assume the following parameters for the sake of the example
NUM_WEEKS = 3
NUM_DAYS_PER_WEEK = 5
NUM_HOURS_PER_DAY = 4
NUM_CLASSROOMS = 30
POPULATION_SIZE = 50
NUM_GENERATIONS = 20
CROSSOVER_RATE = 0.7
MUTATION_RATE = 0.9


# Example classes to represent the data
class Course:
    def __init__(self, code, name):   #constructor jo object ki initial state k setup krta ha by assigning values to its attributes.
        self.code = code   #assigning values
        self.name = name

class Student:
    def __init__(self, name, courses):   #constructor jo object ki initial state k setup krta ha by assigning values to its attributes.
        self.name = name
        self.courses = courses

class Teacher:
    def __init__(self, name):      #constructor jo object ki initial state k setup krta ha by assigning values to its attributes.
        self.name = name

# Initialize courses, students, and teachers
#iterros dataframe ha jis se hum iterate krrhe h
courses = [Course(row['Course Code'], row['Course Name']) for _, row in courses_df.iterrows()]    #yahan humne dataframe use kiya ha jis ki help se hum hr row mei se course ka code or name uthaen gy or usey new object course mei append krengy.
students = {}   #empty dictionary create for students
for index, row in student_course_df.iterrows():   # same wohi dataframe use kr k hum student ka name or course extract krhe ha.
    student_name = row["Student Name"]
    course_code = row["Course Code"]
    if student_name in students:    #yhn check lgya ha k agr to student phly se he dictionary mei ha to phir uske corresponding course ko append krdo.
        students[student_name].append(course_code)
    else:             #agr nhi to phir dono ko dictionary mei append kro.
        students[student_name] = [course_code]

# Remove students with less than three courses
students = {name: courses for name, courses in students.items() if len(courses) >= 3}   #ye line filter krrhi ha k students wali dictionary mei kon se students ha jo less then 3 courses mei enrolled ha
teachers = [Teacher(row['Names']) for index, row in teachers_df.iterrows()]  #dataframe k zariye hum teachers k names iterate krrhe ha or usey new teachers wali dictionary  mei append krrhe ha

# Initialize a population of exam schedules
def create_schedule():
    schedule = {}    #schedule ki aik empty dictionary create ki ha
    scheduled_times = set()  # set to store scheduled exam times
   
#hum course wali dictonary pr iterate krrhe ha or random parameters generate krrhe ha kuch ki range bhi ha jin mei hum random values generate krrhe ha.
#ye loop tb tk iterate kregi jb tk valid and available time slot current course ka generate na hojae.
    for course in courses:
        while True:  
            teacher = random.choice(teachers).name
            classroom = random.randint(301, 330)
            week = random.randint(0, NUM_WEEKS - 1)
            day = random.randint(0, NUM_DAYS_PER_WEEK - 1)
            hour = random.randint(0, NUM_HOURS_PER_DAY - 1)

            # check if this time slot is already scheduled for another course
            if (week, day, hour) not in scheduled_times:
                schedule[course.code] = {
                    'teacher': teacher,
                    'classroom': classroom,
                    'week': week,
                    'day': day,
                    'hour': hour
                }
                scheduled_times.add((week, day, hour))   #jo parameters generate kiye ha ab unhe dictionary mei append krna ha
                break

    return schedule  #in the end schedule ko return krna ha
#population_size basically k hum kitne schedules generate krna chahty ha
#simply hum n aik loop lgae ha jo k population size pr depend krrhi ha jo craete schedule wale function ko call kr k schdeules generate kregi or unhe new list population mei append krengi.
population = [create_schedule() for i in range(POPULATION_SIZE)]



def check_constraints(schedule):
    ###########################Hard Constarints##############################################
    print("\n")
    print("\033[1;4;35mHard Constraints:\033[0m")

    student_courses_count = {}  # Dictionary to store the count of courses for each student
    student_exams = {}  # Dictionary to store exam times for each student

    # Constraint 1: An exam must be scheduled for each course
    for student_name, courses in students.items():
        for course_code in courses:
            if course_code in schedule:  # Check if the course is scheduled
                student_courses_count[student_name] = student_courses_count.get(student_name, 0) + 1  #aik dictionary jo store krrhi ha students ka course count.agr name  dictionary mei nhi ha to wo zero return krega.other wsie one ka increment hga.

    # Constraint 2: a)Check if each student is enrolled in at least 3 courses
    for student_name, count in student_courses_count.items():
        if count < 3:
            print(f"\033[95mStudent ({student_name}) is enrolled in less than 3 courses: {is_enrolled_less_than_3}\033[0m")
            return False
    print("\033[93mStudent is enrolled in less than 3 courses: True\033[0m")
   
    # Constraint 2:b) A student cannot give more than 1 exam at a time
    for course, exam_time in schedule.items():
        for student_name, courses in students.items():
            if course in courses:
                if student_name in student_exams:
                    if exam_time['hour'] == student_exams[student_name]['hour'] and \
                       exam_time['day'] == student_exams[student_name]['day'] and \
                       exam_time['week'] == student_exams[student_name]['week']:
                        print(f"\033[95mA student ({student_name}) cannot give more than 1 exam at a time: False\033[0m")
                        return False
                #aik dictionary create ki ha jis mei students names k coreesponding uak exam schedule as value append krrhe ha.
                student_exams[student_name] = {'hour': exam_time['hour'], 'day': exam_time['day'], 'week': exam_time['week']}
    print("\033[92mA student cannot give more than 1 exam at a time: True\033[0m")
   
    # Constraint 3: A teacher cannot invigilate two exams at the same time
    teacher_exams = {}   #exam k time invigilation k liye jo teacher honngy uski aik dictionary create ki ha
    for course, exam_time in schedule.items():
        teacher_name = exam_time['teacher']
        if teacher_name in teacher_exams:
            if exam_time['hour'] == teacher_exams[teacher_name]['hour'] and \
               exam_time['day'] == teacher_exams[teacher_name]['day'] and \
               exam_time['week'] == teacher_exams[teacher_name]['week']:
                print("\033[95mA teacher cannot invigilate two exams at the same time: False\033[0m")
                return False
        teacher_exams[teacher_name] = {'hour': exam_time['hour'], 'day': exam_time['day'], 'week': exam_time['week']}
    print("\033[94mA teacher cannot invigilate two exams at the same time: True\033[0m")  

    # Constraint 4: A teacher cannot invigilate two exams in a row
    teacher_exams = {}
    for course, exam_time in schedule.items():
        teacher_name = exam_time['teacher']
        if teacher_name in teacher_exams:
            if (abs(exam_time['hour'] - teacher_exams[teacher_name]['hour']) == 1 or abs(exam_time['hour'] - teacher_exams[teacher_name]['hour']) == -1) and \
               exam_time['day'] == teacher_exams[teacher_name]['day'] and \
               exam_time['week'] == teacher_exams[teacher_name]['week']:
                print("\033[95mA teacher cannot invigilate two exams in a row: False\033[0m")
                return False
        teacher_exams[teacher_name] = {'hour': exam_time['hour'], 'day': exam_time['day'], 'week': exam_time['week']}
               
    print("\033[91mA teacher cannot invigilate two exams in a row: True\033[0m")


    # Constraint 5: Exam will not be held on weekends.
    print("\033[95mA Exam will not be held on weekends. : True\033[0m")

    # Constraint 6: Each exam must be held between 9 am and 5 pm.
    print("\033[90mA Each exam must be held between 9 am and 5 pm.: True\033[0m")




    ########################### Soft Constarints##############################################
    print("\n")
    print("\033[1;4;35mSoft Constraints:\033[0m")
    soft_con1=False
    soft_con2=False
    soft_con3=False
    soft_con4=False
    
    
    
    

    total_soft=0      #variable jo count krega k kitne soft constratints full fill hue ha
   
    #Constarint1: All students and teachers shall be given a break on Friday from 1-2.
    sum1 = 0
    for course, exam_time in schedule.items():
        if exam_time['day'] == 4 and exam_time['hour'] == 2:  # Friday, slot 2
            sum1 += 1
            break

    if sum1 > 0:
        total_soft += 1
        soft_con1 = True
    #else:
        #print("\033[93mAll students and teachers shall be given a break on Friday from 1-2.:false\033[0m")

   
   
               
               
      #Constarint2          
     #A student shall not give more than 1 exam consecutively.
    sum2 = 0
    student_exams = {}
    for course, exam_time in schedule.items():
        for student_name, courses in students.items():
            if course in courses:
                if student_name in student_exams:
                    prev_exam = student_exams[student_name]
                    if prev_exam['week'] == exam_time['week'] and prev_exam['day'] == exam_time['day'] and prev_exam['hour'] == exam_time['hour'] - 1:
                        sum2 += 1
                        break
                student_exams[student_name] = exam_time

    if sum2 == 0:
        total_soft += 1
        soft_con2 = True
    #else:
        #print("\033[94mA student shall not give more than 1 exam consecutively.:false\033[0m")

   
   
   
             
      #constarits3      
     #If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam.
    sum3 = 0
    for student_name, courses in students.items():
        mg_course = None
        cs_course = None
        for course in courses:
            if course.startswith("MG"):
                mg_course = course
            elif course.startswith("CS"):
                cs_course = course
        if mg_course and cs_course:
            if schedule[mg_course]['week'] > schedule[cs_course]['week']:
                sum3 += 1
                break
            elif schedule[mg_course]['week'] == schedule[cs_course]['week']:
                if schedule[mg_course]['day'] > schedule[cs_course]['day']:
                    sum3 += 1
                    break
                elif schedule[mg_course]['day'] == schedule[cs_course]['day']:
                    if schedule[mg_course]['hour'] > schedule[cs_course]['hour']:
                        sum3 += 1
                        break

    if sum3 == 0:
        total_soft += 1
        soft_con3 = True
    
    #else:
        #print("\033[92mIf a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam.:false\033[0m")
    
    # Soft Constraint 4: 1 slot hour of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot
    sum4 = 0
    break_count = 0
    for course, exam_time in schedule.items():
        if exam_time['hour'] == 1:  # Considering one slot for break
            break_count += 1

    if break_count == 1:
        total_soft += 1
        soft_con4 = True
    #else:
        #print("\033[92mhour of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot:false\033[0m")






    

    if total_soft < 3 :
       print("\033[1;31mPenalties exceed threshold. Less than three soft constraints are true\033[0m")
       print(total_soft)
       return False
    else:
      print("\n")
      print("\033[96m********************************************\033[0m")
      print("\033[96mSoft constraints fulfilled are the following:\033[0m")
      print("\033[96m********************************************\033[0m")

      if soft_con1:
          print("\033[93mAll students and teachers have a break on Friday from 1-2.\033[0m")
      if soft_con2:
         print("\033[94mA student shall not give more than 1 exam consecutively\033[0m")
      if soft_con3:
         print("\033[92mIf a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam.\033[0m")    
      if soft_con4:
         print("\033[90mTwo hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot.\033[0m")      
   

    return True



def fitness(schedule,i):   #constraint based fitness
   
   
    print("\033[96m" + "----------------------------------------------" + "\033[0m")
    print("\033[96m" + " " * ((38 - len(f"Population: {i}")) // 2) + f"Population: {i}" + " " * ((38 - len(f"Population: {i}")) // 2) + "\033[0m")
    print("\033[96m" + "----------------------------------------------" + "\033[0m")


    conflicts = 0
    constraints=check_constraints(schedule)
    if constraints:
       conflicts = 1
       print("\n")
       print("\033[1;35m*******All constraints fulfilled....*******\033[0m")
       print("\n")

    # ... (rest of fitness function)
    else:
        conflicts = 100  # Assign a high penalty for infeasible schedules

    return conflicts

# Genetic algorithm operations
def selection(population, fitness_values):#roulette wheel selection
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected_parents = []
    for _ in range(2):
        spin = random.random()
        cumulative_probability = 0
        for i, probability in enumerate(probabilities):
            cumulative_probability += probability
            if spin <= cumulative_probability:
                selected_parents.append(population[i])
                break
    return selected_parents
 
   
def crossover(parents):  #single -point crossover
    if random.random() < CROSSOVER_RATE:
        crossover_point = random.randint(1, len(courses) - 1)
        child1 = {}
        child2 = {}
        courses_list = list(parents[0].keys())
        for idx, course in enumerate(courses_list):
            if idx < crossover_point:
                child1[course] = parents[0][course]
                child2[course] = parents[1][course]
            else:
                child1[course] = parents[1][course]
                child2[course] = parents[0][course]
        return [child1, child2]
    return parents

def mutation(schedule):  #random mutation
    if random.random() < MUTATION_RATE:
        unscheduled_courses = [course for course in courses if course.code not in schedule]
        if unscheduled_courses:
            course_to_schedule = random.choice(unscheduled_courses)
            schedule[course_to_schedule.code] = {
                'teacher': random.choice(teachers).name,
                'classroom': random.randint(301, 330),
                'week': random.randint(0, NUM_WEEKS - 1),
                'day': random.randint(0, NUM_DAYS_PER_WEEK - 1),
                'hour': random.randint(0, NUM_HOURS_PER_DAY - 1)
              }
    return schedule







for generation in range(NUM_GENERATIONS):
    print("\n")
    print("\033[0;33m" + "\u2588" * 20)  # Dark yellow color and left border
    print(f"\033[0;33m\u2588   Generation {generation+1}   \u2588")  # Centered message within dark yellow borders
    print("\u2588" * 20 + "\033[0m")  # Right border and reset text formatting
    #fitness_values = [fitness(schedule,i++) for schedule in population]
    i=1

    
    fitness_values = []
    for schedule in population:
        fitness_values.append(fitness(schedule, i))
        i += 1
    best_schedule_idx = np.argmin(fitness_values)
    best_schedule = population[best_schedule_idx]
   
    print("\n")
    print("\033[0;33m" + f"*****Best Fitness*****: {min(fitness_values)}\n\n" + "\033[0m")
    if min(fitness_values) == 1:  # If the best fitness is already 1, all constraints are fulfilled
        best_schedule = population[best_schedule_idx]
        break
   
    new_population = []
    while len(new_population) < POPULATION_SIZE:
        parents = selection(population, fitness_values)
        children = crossover(parents)
        mutated_children = [mutation(child) for child in children]
        new_population.extend(mutated_children)

    population = new_population
   
best_schedule = population[0] if population else None
best_fitness = fitness_values[0] if fitness_values else None

for i in range(1, len(population)):
    if fitness_values[i] < best_fitness:
        best_fitness = fitness_values[i]
        best_schedule = population[i]

#print("Best Schedule:")
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Define time slots
time_slots = ['9 am - 11 am', '11 am - 1 pm', '1 pm - 3 pm', '3 pm - 5 pm']

"""schedule_data = []

# Populate the list with course information
for course in courses:
    course_code = course.code
    course_name = course.name
    if course_code in best_schedule:
        info = best_schedule[course_code]
        schedule_data.append([course_code, course_name, info['teacher'], info['classroom'], weekdays[info['day']], time_slots[info['hour']], info['week']+1])
    else:
        schedule_data.append([course_code, course_name, 'Not scheduled', 'Not scheduled', 'Not scheduled', 'Not scheduled', 'Not scheduled'])

# Create a DataFrame from the schedule data
schedule_df = pd.DataFrame(schedule_data, columns=['Course Code', 'Course Name', 'Teacher', 'Classroom', 'Day', 'Hour', 'Week'])

# Display the DataFrame as a table
display(schedule_df)"""
teacher_schedule_data = []

for teacher in teachers:
    teacher_name = teacher.name
    teacher_courses = [course for course, info in best_schedule.items() if info['teacher'] == teacher_name]
    for course_code in teacher_courses:
        info = best_schedule[course_code]
        teacher_schedule_data.append([teacher_name, course_code, courses_df.loc[courses_df['Course Code'] == course_code, 'Course Name'].iloc[0], weekdays[info['day']], time_slots[info['hour']], info['week']+1])

# Create a DataFrame for teacher schedule
teacher_schedule_df = pd.DataFrame(teacher_schedule_data, columns=['Teacher', 'Course Code', 'Course Name', 'Day', 'Hour', 'Week'])

print("\033[4;35m_________________\033[0m")
print("\033[1;3;35m\nTeacher Schedule:\033[0m")
print("\033[4;35m_________________\033[0m")

display(teacher_schedule_df)


# Create a list to store student schedule data
student_schedule_data = []

for student_name, student_courses in students.items():
    for course_code in student_courses:
        if course_code in best_schedule:
            info = best_schedule[course_code]
            student_schedule_data.append([student_name, course_code, courses_df.loc[courses_df['Course Code'] == course_code, 'Course Name'].iloc[0], info['teacher'], weekdays[info['day']], time_slots[info['hour']], info['week']+1])

# Create a DataFrame for student schedule
student_schedule_df = pd.DataFrame(student_schedule_data, columns=['Student Name', 'Course Code', 'Course Name', 'Teacher', 'Day', 'Hour', 'Week'])

# Display the student schedule DataFrame as a table
print("\n")
print("\033[4;35m_________________\033[0m")
print("\033[1;3;35m\nStudent Schedule:\033[0m")
print("\033[4;35m_________________\033[0m")


pd.set_option('display.max_rows', None)
display(student_schedule_df)



████████████████████
█   Generation 1   █
████████████████████
----------------------------------------------
            Population: 1            
----------------------------------------------


Hard Constraints:
Student is enrolled in less than 3 courses: True
A student cannot give more than 1 exam at a time: True
A teacher cannot invigilate two exams at the same time: True
A teacher cannot invigilate two exams in a row: False
----------------------------------------------
            Population: 2            
----------------------------------------------


Hard Constraints:
Student is enrolled in less than 3 courses: True
A student cannot give more than 1 exam at a time: True
A teacher cannot invigilate two exams at the same time: True
A teacher cannot invigilate two exams in a row: True
A Exam will not be held on weekends. : True
A Each exam must be held between 9 am and 5 pm.: True


Soft Constraints:
Penalties exceed threshold. Less than three soft constraints are true
1
----

,Teacher,Course Code,Course Name,Day,Hour,Week
0,Farah Naz,CS217,Object Oriented Programming,Tuesday,3 pm - 5 pm,3
1,Hamda Khan,CY2012,Digital Forensics,Monday,1 pm - 3 pm,1
2,Usman Rashid,SS113,Pakistan Studies,Wednesday,3 pm - 5 pm,1
3,Sara Aziz,SS118,Psychology,Friday,3 pm - 5 pm,3
4,Maimoona Rassol,DS3011,Big Data Analytics,Friday,1 pm - 3 pm,2
5,Sajid Khan,CS218,Data Structures,Thursday,1 pm - 3 pm,3
6,Shahzad Mehmood,SE110,Intro to Software Engineering,Monday,11 am - 1 pm,1
7,Sanaa Ilyas,SS111,Islamic and Religious Studies,Friday,1 pm - 3 pm,1
8,Nagina Safdar,CS220,Operating Systems,Wednesday,1 pm - 3 pm,2
9,Usman Ashraf,CS118,Programming Fundamentals,Thursday,3 pm - 5 pm,1




_________________

Student Schedule:
_________________


,Student Name,Course Code,Course Name,Teacher,Day,Hour,Week
0,Sam D Edwards,AI2011,Programming for AI,Shoaib Mehboob,Tuesday,3 pm - 5 pm,1
1,Sam D Edwards,SS152,Communication & Presentation Skills,Zainab Abaid,Monday,3 pm - 5 pm,2
2,Sam D Edwards,EE229,Computer Organization and Assembly Language,Farwa Batool,Tuesday,11 am - 1 pm,3
3,Sheila Hughton,DS3011,Big Data Analytics,Maimoona Rassol,Friday,1 pm - 3 pm,2
4,Sheila Hughton,MG220,Marketing Management,Hammad Majeed,Wednesday,9 am - 11 am,1
5,Sheila Hughton,CS118,Programming Fundamentals,Usman Ashraf,Thursday,3 pm - 5 pm,1
6,Yasmin Ahmed,SE110,Intro to Software Engineering,Shahzad Mehmood,Monday,11 am - 1 pm,1
7,Yasmin Ahmed,CS307,Computer Networks,Hasan Mujtaba,Friday,1 pm - 3 pm,3
8,Yasmin Ahmed,SS113,Pakistan Studies,Usman Rashid,Wednesday,3 pm - 5 pm,1
9,Sarah N Md Sallehuddin Khan,EE229,Computer Organization and Assembly Language,Farwa Batool,Tuesday,11 am - 1 pm,3
